# Task 1: 
Steven is a travel blogger that intends to create a travel food series. He is looking at data from Zomato for inspiration. He wants to find restaurants that have good user ratings and interesting past events.
Steven obtained the following data:

·   	List of restaurants

·   	Country Code (in excel format)

In [1]:
import requests
import json
import pandas as pd

#load list of restaurants
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
response = requests.get(url)
data = json.loads(response.text)
#print(data)

#load xlsx file
excel_file_path = 'Country-Code.xlsx'
country_codes_df = pd.read_excel(excel_file_path)
country_codes_df.columns = ['country_code', 'country']
country_codes_df.head()

/Users/bryanlee/Library/Python/3.9/lib/python/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


,country_code,country
0,1,India
1,14,Australia
2,30,Brazil
3,37,Canada
4,94,Indonesia


## 1. 	Extract the following fields and store the data as restaurants.csv.

◦   	Restaurant Id

◦   	Restaurant Name

◦   	Country

◦   	City

◦   	User Rating Votes

◦   	User Aggregate Rating (in float)

◦   	Cuisines

__Steps__
1. Inspect and extract relevant information from the json
2. Putting it into a dataframe
3. Joining with the xlsx data to get the Country
4. Format and store as csv

In [10]:
#1.
restaurant_in_list = []
for restaurant in data[0]['restaurants']:
    entry = restaurant['restaurant']
    restaurant_in_list.append({
        'restaurant_id': restaurant['restaurant']['R']['res_id'],
        'restaurant_name': restaurant['restaurant']['name'],
        'country_id': restaurant['restaurant']['location']['country_id'],
        'city': restaurant['restaurant']['location']['city'],
        'user_rating_votes': restaurant['restaurant']['user_rating']['votes'],
        'user_aggregate_rating': float(restaurant['restaurant']['user_rating']['aggregate_rating']),
        'cuisines': restaurant['restaurant']['cuisines']
    })
    
#2.
restaurant_in_df = pd.DataFrame(restaurant_in_list)
print(restaurant_in_df.head())

#3.
final_restaurant_df = restaurant_in_df.merge(country_codes_df, how='left', left_on='country_id', right_on='country_code')

#4.
final_columns = ['restaurant_id', 'restaurant_name', 'country', 'city', 'user_rating_votes', 
                 'user_aggregate_rating', 'cuisines']
final_restaurant_df = final_restaurant_df[final_columns]
final_restaurant_df.to_csv('restaurants.csv', index=False)

   restaurant_id       restaurant_name  country_id       city  \
0       18649486  The Drunken Botanist           1    Gurgaon   
1         308322      Hauz Khas Social           1  New Delhi   
2       18856789  AIR- An Ivory Region           1  New Delhi   
3         307374              AMA Cafe           1  New Delhi   
4       18238278               Tamasha           1  New Delhi   

  user_rating_votes  user_aggregate_rating  \
0              4765                    4.4   
1             13627                    4.6   
2              1819                    4.1   
3              3252                    4.4   
4              8112                    4.4   

                                            cuisines  
0        Continental, Italian, North Indian, Chinese  
1  Continental, American, Asian, North Indian, Ch...  
2          North Indian, Chinese, Continental, Asian  
3                                       Cafe, Juices  
4    Finger Food, North Indian, Continental, Italian  
